In [13]:
import sys, os
import numpy as np
import faiss
import torch
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from tqdm import tqdm

Set folder path

In [2]:
DATABASE_PATH = os.path.abspath("../../var/database")
DOCS_PATH = os.path.abspath("../../docs")
DEVICE = "cpu"

# Read model name
with open(os.path.join(DATABASE_PATH, "model.txt"), "r", encoding="utf-8") as f:
    EMBEDDINGS_MODEL_NAME = f.read()

In [3]:
def get_embedding():
    
    """
    Returns embedding
    """
    
    embeddings_model = HuggingFaceEmbeddings(
        model_name=EMBEDDINGS_MODEL_NAME,
        model_kwargs={"device": DEVICE}
    )
    vector_store = FAISS.load_local(
        DATABASE_PATH,
        embeddings=embeddings_model,
        allow_dangerous_deserialization=True
    )
    return embeddings_model, vector_store


embeddings_model, vector_store = get_embedding()

In [14]:
llm = ChatOllama(
    base_url="http://database_ollama:11434",
    model="qwen2.5:1.5b",
    temperature = 0,
)

In [ ]:
def find_docs(questions):
    
    """
    Find documents by questions
    """
    
    embeddings = np.array(embeddings_model.embed_documents(questions))
    mean_embedding = embeddings.mean(axis=0).tolist()
    docs = vector_store.similarity_search_by_vector(mean_embedding, k=3)
    return embeddings, docs

def print_docs(docs):
    for i, doc in enumerate(docs):
        print(f"Result {i+1}:")
        print(f"ID: {doc.metadata['id']}")
        print(f"Content:\n{doc.page_content}")
        #print(f"Distance: {distance}")
        if i + 1 < len(docs):
            print("-" * 50)


def get_llm_question(llm, question, history):
        
    prompt = [
        ("system",
            "Ты помощник, который делает вопросы самодостаточными, добавляя контекст из истории диалога"
        ),
        ("human",
            f"История диалога:\n\n" + "\n".join(history) + "\n\n" +
            f"Переформулируй вопрос, на основе истории диалога, " +
                f"чтобы он был понятен без контекста: {question}. Вопрос:"
        ),
    ]
    
    print(str(prompt) + "\n")
    
    answer = llm.invoke(prompt).content
    
    print("Новый вопрос: " + answer + "\n")
    
    return answer


def get_context1(question, history):
    questions = history + [question]
    
    # Ищем релевантные документы
    _, docs = find_docs(questions)
    print_docs(docs)


def get_context2(question, history):
    
    new_question = question
    
    # Переформулировка вопроса
    if len(history) > 0:
        new_question = get_llm_question(llm, question, history)
    
    # Ищем релевантные документы
    _, docs = find_docs([new_question])
    print_docs(docs)


history = [
    "Что такое BayLang?"
]
question = "Как его установить?"
get_context2(question, history)